### This project aims at exploring, cleaning and deriving the following analytics from the Youtube Trending Dataset
### 1. Most popular Channels<br> 2. Most Viewed Videos <br> 3. Most popular Videos <br> 4. Most liked Videos<br> 5. Most disliked Videos
The dataset can be downloaded from Kaggle(https://www.kaggle.com/datasnaek/youtube-new?select=CAvideos.csv)

Various steps involved in this project
1. Setting up Spark session
2. Look at the data and number of columns
3. Explore and Clean Data
    i.   Identified duplicates --> dropped duplicates present in the data
    ii.  Analyzed all the columns and converted datatype to perform analytics
    iii. Handled Null Values
4. Generate Analytics    

#### Setting up Spark Session

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [2]:
configure = SparkConf().setAppName('YoutubeTrending').setMaster('local')

In [3]:
sc = SparkContext(conf = configure)

In [4]:
spark = SparkSession.builder.appName('YoutubeTrending').getOrCreate()

In [6]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.name', 'YoutubeTrending'),
 ('spark.driver.host', '10.225.144.245'),
 ('spark.driver.port', '46263'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1598768534213')]

#### Reading Data

In [7]:
file_path = 'Data/Youtube Dataset/CAvideos.csv'
infer_schema = 'true'
file_type = 'csv'
include_header = 'true'
delimiter = ','

In [8]:
df = spark.read.format(file_type) \
        .option("InferSchema", infer_schema) \
        .option("header", include_header) \
        .option("delimiter", delimiter) \
        .load(file_path)
        

In [9]:
df.show(5)

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|n1WpP7iowLc|     17.14.11|Eminem - Walk On ...|   EminemVEVO|         10|2017-11-10T17:00:...|"Eminem"|"Walk"|"...|17158579| 787425|   43420|       125882|https://i.ytimg.c...|            False|           False|                 

In [10]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [11]:
df.count()

45560

In [12]:
len(df.schema.names)

16

It can be observed that there are a total of 45560 rows, 16 columns in the given dataset. 
There seems to be null values in all the columns. We will have to fill null values with appropriate replacements

#### Data Exploration and Data Cleaning

In [13]:
for col in df.schema.names:
    df.describe(col).show()

+-------+----------------------------+
|summary|                    video_id|
+-------+----------------------------+
|  count|                       45560|
|   mean|                        null|
| stddev|                        null|
|    min|                      #DRAMA|
|    max|◎追蹤【三立新聞網Instagra...|
+-------+----------------------------+

+-------+--------------------+
|summary|       trending_date|
+-------+--------------------+
|  count|               41537|
|   mean|              1225.2|
| stddev|  1085.5861089752393|
|    min| 1992 and 1994 Ga...|
|    max|” “Gorillas in th...|
+-------+--------------------+

+-------+--------------------+
|summary|               title|
+-------+--------------------+
|  count|               41383|
|   mean|               264.0|
| stddev|  213.27447104611466|
|    min|                  36|
|    max|🚨Active Shooter ...|
+-------+--------------------+

+-------+-------------+
|summary|channel_title|
+-------+-------------+
|  count|        41321|
|

In [14]:
# Drop duplicates from the dataset
if df.drop_duplicates().count() > 0:
    df = df.drop_duplicates()

In [15]:
df.count()

41686

##### Converting Datatypes for mathematical operations

In [16]:
# [x.views for x in df.select('views').distinct().collect()]

In [17]:
# [x.comment_count for x in df.select('comment_count').distinct().collect()]

In [18]:
# [x.likes for x in df.select('likes').distinct().collect()]

In [19]:
# [x.dislikes for x in df.select('dislikes').distinct().collect()]

In [20]:
# for col in df.schema.names:
#     print(col)
#     print()
#     print([x[col] for x in df.select(col).distinct().collect()])
#     print()

In [21]:
df = df.withColumn('views', df.views.cast(T.IntegerType()))
df = df.withColumn('dislikes', df.dislikes.cast(T.IntegerType()))
df = df.withColumn('likes', df.likes.cast(T.IntegerType()))
df = df.withColumn('comment_count', df.comment_count.cast(T.IntegerType()))

In [22]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



Thus, we have looked for any values with non-descrepancies and then converted the datatypes to integertype

##### Handling Null Values

In [24]:
print('Column', '|', 'Number_of_Null_Values')
for col in df.schema.names:    
    print(col, '|', df.filter((df[col] == "") | df[col].isNull() | F.isnan(df[col])).count())

Column | Number_of_Null_Values
video_id | 0
trending_date | 577
title | 616
channel_title | 639
category_id | 678
publish_time | 699
tags | 777
views | 805
likes | 805
dislikes | 805
comment_count | 805
thumbnail_link | 792
comments_disabled | 794
ratings_disabled | 796
video_error_or_removed | 799
description | 2095


In [25]:
df = df.filter((df['video_id'] != "") | df['video_id'].isNotNull())
df.count()

41686

In [26]:
df = df.filter((df['likes'] != "") | df['likes'].isNotNull())
df.count()

40881

In [27]:
df = df.filter((df['dislikes'] != "") | df['dislikes'].isNotNull())
df.count()

40881

In [28]:
df = df.filter((df['views'] != "") | df['views'].isNotNull())
df.count()

40881

In [29]:
df = df.filter((df['title'] != "") | df['title'].isNotNull())
df.count()

40881

In [30]:
df = df.filter((df['comment_count'] != "") | df['comment_count'].isNotNull())
df.count()

40881

Thus, we have identified all the null values and filtered our dataset for the columns having null values

#### Analytics

##### Most liked Videos

In [60]:
most_liked_videos = df\
.groupBy('title')\
.agg(F.sum('likes'))\
.withColumnRenamed("sum(likes)", "total_num_likes")\
.sort(F.desc("total_num_likes"))

In [61]:
most_liked_videos.show(10, truncate=False)

+----------------------------------------------------------+---------------+
|title                                                     |total_num_likes|
+----------------------------------------------------------+---------------+
|BTS (방탄소년단) 'FAKE LOVE' Official MV                  |23078610       |
|Marvel Studios' Avengers: Infinity War Official Trailer   |18929848       |
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind        |16687951       |
|Childish Gambino - This Is America (Official Video)       |14593916       |
|Drake - God’s Plan                                        |12164958       |
|Maroon 5 - Girls Like You ft. Cardi B                     |9820458        |
|Ariana Grande - No Tears Left To Cry                      |9696468        |
|Marvel Studios' Avengers: Infinity War - Official Trailer |9466506        |
|BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Official MV|9373576        |
|Taylor Swift - End Game ft. Ed Sheeran, Future            |9282504        |
+--------

##### Most Popular Videos

In [78]:
most_popular_videos = df \
.groupBy('title')\
.agg(F.sum('comment_count'))\
.withColumnRenamed("sum(comment_count)", "total_comments")\
.sort(F.desc("total_comments"))

In [79]:
most_popular_videos.show(10, truncate=False)

+---------------------------------------------------------------------+--------------+
|title                                                                |total_comments|
+---------------------------------------------------------------------+--------------+
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind                   |5087713       |
|BTS (방탄소년단) 'FAKE LOVE' Official MV                             |4838370       |
|Suicide: Be Here Tomorrow.                                           |2768628       |
|Marvel Studios' Avengers: Infinity War Official Trailer              |2556491       |
|BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Official MV           |1894345       |
|Childish Gambino - This Is America (Official Video)                  |1545142       |
|we broke up                                                          |1404418       |
|Marvel Studios' Avengers: Infinity War - Official Trailer            |1190170       |
|CHẠY NGAY ĐI | RUN NOW | SƠN TÙNG M-TP | Official Mu

##### Most disliked videos

In [75]:
most_disliked_videos = df\
.groupBy('title')\
.agg(F.sum('dislikes'))\
.withColumnRenamed("sum(dislikes)", "tot_dislikes")\
.withColumnRenamed("title", "Title")\
.sort(F.desc("tot_dislikes"))

In [76]:
most_disliked_videos.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------+------------+
|Title                                                                                              |tot_dislikes|
+---------------------------------------------------------------------------------------------------+------------+
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind                                                 |8468160     |
|Jake Paul - It's Everyday Bro (Remix) [feat. Gucci Mane]                                           |2117354     |
|Suicide: Be Here Tomorrow.                                                                         |1777418     |
|Fergie Performs The U.S. National Anthem / 2018 NBA All-Star Game                                  |818658      |
|Childish Gambino - This Is America (Official Video)                                                |689948      |
|BTS (방탄소년단) 'FAKE LOVE' Official MV                                            

##### Most viewed videos

In [81]:
most_viewed_videos = df\
.groupBy('title')\
.agg(F.sum('views'))\
.withColumnRenamed('sum(views)', 'total_views')\
.withColumnRenamed('title', 'Title')\
.sort(F.desc('total_views'))

In [82]:
most_viewed_videos.show(10, truncate=False)

+---------------------------------------------------------+-----------+
|Title                                                    |total_views|
+---------------------------------------------------------+-----------+
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind       |631428335  |
|Marvel Studios' Avengers: Infinity War Official Trailer  |577553186  |
|Childish Gambino - This Is America (Official Video)      |411775069  |
|BTS (방탄소년단) 'FAKE LOVE' Official MV                 |321743622  |
|VENOM - Official Trailer (HD)                            |268913813  |
|Marvel Studios' Avengers: Infinity War - Official Trailer|260624189  |
|To Our Daughter                                          |242252692  |
|Luis Fonsi, Demi Lovato - Échame La Culpa                |191071336  |
|Taylor Swift - Delicate                                  |191059970  |
|Drake - God’s Plan                                       |190183754  |
+---------------------------------------------------------+----------

##### Most popular Channels

In [87]:
most_trending_channels = df\
.groupBy('channel_title')\
.agg(F.sum('views'))\
.withColumnRenamed('sum(views)', 'total_views')\
.withColumnRenamed('channel_title', 'Channel')\
.sort(F.desc('total_views'))

In [88]:
most_trending_channels.show(10, truncate=False)

+---------------------------+-----------+
|Channel                    |total_views|
+---------------------------+-----------+
|Marvel Entertainment       |1011420205 |
|T-Series                   |799114025  |
|Dude Perfect               |729916338  |
|YouTube Spotlight          |635976769  |
|ibighit                    |511567918  |
|PewDiePie                  |461700524  |
|Sony Pictures Entertainment|451188760  |
|Speed Records              |426604974  |
|20th Century Fox           |419577035  |
|ChildishGambinoVEVO        |411775069  |
+---------------------------+-----------+
only showing top 10 rows

